In [1]:
pip install jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 30.8 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 54.5 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.45.1
    Uninstalling transformers-4.45.1:
      Successfully uninstalled transformers-4.45.1


In [3]:
import torch
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from transformers import Trainer, TrainingArguments
from transformers import VisionEncoderDecoderModel, AutoProcessor
from datasets import load_dataset
from PIL import Image
import matplotlib.pyplot as plt
import os
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm_notebook as tqdm
from transformers import AdamW
import torch.nn.functional as F
device='cuda' if torch.cuda.is_available() else 'cpu'
from jiwer import wer

In [4]:
class Customdataset(Dataset):
    def __init__(self, li):
        super().__init__()
        self.li=li
    def __len__(self):
        return len(self.li)
    def __getitem__(self,idx):
        path=self.li[idx][0]
        y=self.li[idx][1]
        return {
            'image':path,
            'text':y
        }

In [5]:
li = []
image_path = '/kaggle/input/ai-of-god-3/Public_data/train_images'
csv_path = '/kaggle/input/ai-of-god-3/Public_data/train.csv'

df = pd.read_csv(csv_path)

for index, row in tqdm(df.iterrows()):
    full_path = os.path.join(image_path, row['unique Id'] + '.png')
    if os.path.isfile(full_path):
        li.append([full_path, row['transcription']])

0it [00:00, ?it/s]

In [6]:
data_train,data_val=train_test_split(li,train_size=0.8, random_state=42)

In [7]:
# Load the model and processor
model = VisionEncoderDecoderModel.from_pretrained("qantev/trocr-small-spanish")
processor = AutoProcessor.from_pretrained("qantev/trocr-small-spanish")
tokenizer = TrOCRProcessor.from_pretrained('qantev/trocr-small-spanish')

config.json:   0%|          | 0.00/4.82k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/247M [00:00<?, ?B/s]

VisionEncoderDecoderModel has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


generation_config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/450 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/510 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.49M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

In [8]:
train_data=Customdataset(data_train)
val_data=Customdataset(data_val)

In [20]:
# def preprocess_batch(batch):
#     preprocess_pipeline = transforms.Compose([
#         transforms.Grayscale(num_output_channels=3),  # Convert to grayscale with 3 channels
#         transforms.ToTensor()
#     ])

#     processed_images = []
#     threshold = 0.6875  # Set your threshold

#     # Process each image in the batch
#     for image_path in batch['image']:
#         # Open the image
#         image = Image.open(image_path).convert("RGB")
        
#         # Apply the grayscale transformation
#         grayscale_image = preprocess_pipeline(image)

#         # Convert to numpy for pixel manipulation
#         grayscale_np = grayscale_image.numpy()

#         # Create a mask where pixel values are less than the threshold
#         mask = grayscale_np < threshold

#         # Turn those pixels black (0.0) and the others white (1.0)
#         grayscale_np[mask] = 0.0      # Text will be black
#         grayscale_np[~mask] = 1.0     # Background will be white

#         # Convert back to tensor and add to the list
#         processed_image = torch.tensor(grayscale_np)
#         processed_images.append(processed_image)

#     # Stack processed images into a single tensor (N, C, H, W)
#     pixel_values = torch.cat([processor(image, return_tensors="pt").pixel_values for image in batch['image']], dim=0)

#     # Process text and tokenize labels
#     labels = torch.cat([
#         processor.tokenizer(text, padding="max_length", truncation=True, max_length=128, return_tensors="pt").input_ids
#         for text in batch['text']
#     ], dim=0)

#     return {"pixel_values": pixel_values, "labels": labels}

In [22]:
def preprocess_batch(batch, threshold=0.6875):
    # Define the preprocessing pipeline
    preprocess_pipeline = transforms.Compose([
        transforms.Grayscale(num_output_channels=3),  # Convert image to grayscale with 3 channels
        transforms.ToTensor(),  # Convert image to tensor
    ])

    # Process images in batch
    images = [Image.open(image_path).convert("RGB") for image_path in batch['image']]

    # Parallelize transformation using torch's stack
    grayscale_images = torch.cat([preprocess_pipeline(image).unsqueeze(0) for image in images], dim=0)
    
    grayscale_images = grayscale_images.to(device)

    # Apply thresholding
    mask = grayscale_images < threshold
    grayscale_images[mask] = 0.0  # Black for values below threshold
    grayscale_images[~mask] = 1.0  # White for values above threshold

    # Use the processor to process the images
    pixel_values = processor(grayscale_images, return_tensors="pt", do_rescale=False).pixel_values

    # Tokenize text labels in the batch
    labels = processor.tokenizer(
        batch['text'], padding="max_length", truncation=True, max_length=128, return_tensors="pt"
    ).input_ids

    return {"pixel_values": pixel_values, "labels": labels}

In [11]:
# Define the preprocessing function
def preprocess_image(image_path, threshold=0.6875):
    # Open the image
    image = Image.open(image_path).convert("RGB")

    # Apply the grayscale transformation
    transform = transforms.Compose([
        transforms.Grayscale(num_output_channels=3),  # Convert to grayscale with 3 channels
        transforms.ToTensor()
    ])

    # Apply the transformations
    grayscale_image = transform(image)

    # Convert to numpy for pixel manipulation
    grayscale_np = grayscale_image.numpy()

    # Create a mask where pixel values are less than the threshold
    mask = grayscale_np < threshold

    # Turn those pixels black (0.0) and the others white (1.0)
    grayscale_np[mask] = 0.0      # Text will be black
    grayscale_np[~mask] = 1.0     # Background will be white

    # Convert back to tensor
    processed_image = torch.tensor(grayscale_np)

    return processed_image

In [12]:
train_loader = DataLoader(train_data, batch_size=8, shuffle=True)
val_loader = DataLoader(val_data, batch_size=8, shuffle=True)

In [13]:
import numpy as np

def wer(reference, hypothesis):
    """
    Calculate the Word Error Rate (WER) between a reference and hypothesis sentence.
    WER is defined as the minimum number of word-level edits (insertions, deletions, substitutions)
    required to convert the hypothesis sentence into the reference sentence, divided by the total number of words in the reference.

    Parameters:
    reference (str): The ground truth sentence.
    hypothesis (str): The predicted sentence.

    Returns:
    float: The word error rate between the reference and hypothesis.
    """

    # Split the reference and hypothesis sentences into words
    reference_words = reference.split()
    hypothesis_words = hypothesis.split()

    # Create a matrix to store the distances
    d = np.zeros((len(reference_words) + 1, len(hypothesis_words) + 1), dtype=np.uint8)

    # Initialize the matrix
    for i in range(1, len(reference_words) + 1):
        d[i][0] = i
    for j in range(1, len(hypothesis_words) + 1):
        d[0][j] = j

    # Fill the matrix
    for i in range(1, len(reference_words) + 1):
        for j in range(1, len(hypothesis_words) + 1):
            if reference_words[i - 1] == hypothesis_words[j - 1]:
                d[i][j] = d[i - 1][j - 1]
            else:
                d[i][j] = min(d[i - 1][j], d[i][j - 1], d[i - 1][j - 1]) + 1

    # The WER is the number of errors divided by the total number of words in the reference sentence
    wer_value = d[len(reference_words)][len(hypothesis_words)] / len(reference_words)

    return wer_value

In [14]:
def avg_wer(ground_truths, predictions):
    wer_loss=0
    for reference, hypothesis in zip(ground_truths, predictions):
        wer_loss += wer(reference, hypothesis)
    return wer_loss/len(ground_truths)

In [15]:
model = model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

num_epochs = 14

In [16]:
LR_SCHEDULER = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)

In [17]:
torch.cuda.empty_cache()

In [18]:
# for epoch in range(num_epochs):
#     print(f"Epoch {epoch + 1}/{num_epochs}")

#     model.train()
    
#     predictions = []
#     ground_truths = []

#     for batch in tqdm(train_loader, desc="Training Batches"):
#         processed_batch = preprocess_batch(batch)
        
#         pixel_values = processed_batch['pixel_values'].to(device)
#         labels = processed_batch['labels'].to(device)

#         outputs = model(pixel_values=pixel_values, labels=labels)
        
#         predicted_ids = outputs.logits.argmax(dim=-1)
#         predicted_strings = tokenizer.batch_decode(predicted_ids, skip_special_tokens=True)

#         true_ids = labels.detach().cpu().numpy()
#         true_strings = tokenizer.batch_decode(true_ids, skip_special_tokens=True)

#         predictions.extend(predicted_strings)
#         ground_truths.extend(true_strings)

#         loss = outputs.loss

#         loss.backward()
#         optimizer.step()
#         optimizer.zero_grad()

#     avg_train_wer_loss = avg_wer(ground_truths, predictions)
#     print(f"Average Training WER Loss: {avg_train_wer_loss:.4f}")
#     LR_SCHEDULER.step()
    
#     predictions = []
#     ground_truths = []

#     model.eval()

#     with torch.no_grad():
#         for batch in tqdm(val_loader, desc="Validation Batches"):
#             processed_batch = preprocess_batch(batch)
#             pixel_values = processed_batch['pixel_values'].to(device)
#             labels = processed_batch['labels'].to(device)

#             outputs = model(pixel_values=pixel_values, labels=labels)
            
#             predicted_ids = outputs.logits.argmax(dim=-1)
#             predicted_strings = tokenizer.batch_decode(predicted_ids, skip_special_tokens=True)

#             true_ids = labels.detach().cpu().numpy()
#             true_strings = tokenizer.batch_decode(true_ids, skip_special_tokens=True)

#             predictions.extend(predicted_strings)
#             ground_truths.extend(true_strings)

#     avg_val_wer_loss = avg_wer(ground_truths, predictions)
#     print(f"Average Validation WER Loss: {avg_val_wer_loss:.4f}")

In [23]:
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")

    # Set the model to training mode
    model.train()

    predictions = []
    ground_truths = []

    for batch in tqdm(train_loader, desc="Training Batches"):
        # Preprocess the batch (apply transformations and tokenization)
        processed_batch = preprocess_batch(batch)

        # Move pixel values and labels to the GPU
        pixel_values = processed_batch['pixel_values'].to(device)
        labels = processed_batch['labels'].to(device)

        # Perform forward pass (compute logits and loss)
        outputs = model(pixel_values=pixel_values, labels=labels)

        # Get the predicted token IDs
        predicted_ids = outputs.logits.argmax(dim=-1)

        # Decode the predicted token IDs into strings
        predicted_strings = tokenizer.batch_decode(predicted_ids, skip_special_tokens=True)

        # Decode the ground truth labels into strings
        true_ids = labels.detach().cpu().numpy()
        true_strings = tokenizer.batch_decode(true_ids, skip_special_tokens=True)

        # Collect the predictions and ground truths
        predictions.extend(predicted_strings)
        ground_truths.extend(true_strings)

        # Compute loss and backpropagate
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    # Calculate and print the Average Word Error Rate (WER) loss for training
    avg_train_wer_loss = avg_wer(ground_truths, predictions)
    print(f"Average Training WER Loss: {avg_train_wer_loss:.4f}")

    # Update the learning rate scheduler
    LR_SCHEDULER.step()

    # Validation Phase
    predictions = []
    ground_truths = []

    # Set the model to evaluation mode
    model.eval()

    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Validation Batches"):
            # Preprocess the batch for validation
            processed_batch = preprocess_batch(batch)

            # Move pixel values and labels to the GPU
            pixel_values = processed_batch['pixel_values'].to(device)
            labels = processed_batch['labels'].to(device)

            # Perform forward pass (compute logits, no backpropagation)
            outputs = model(pixel_values=pixel_values, labels=labels)

            # Get the predicted token IDs
            predicted_ids = outputs.logits.argmax(dim=-1)

            # Decode the predicted token IDs into strings
            predicted_strings = tokenizer.batch_decode(predicted_ids, skip_special_tokens=True)

            # Decode the ground truth labels into strings
            true_ids = labels.detach().cpu().numpy()
            true_strings = tokenizer.batch_decode(true_ids, skip_special_tokens=True)

            # Collect the predictions and ground truths
            predictions.extend(predicted_strings)
            ground_truths.extend(true_strings)

    # Calculate and print the Average WER loss for validation
    avg_val_wer_loss = avg_wer(ground_truths, predictions)
    print(f"Average Validation WER Loss: {avg_val_wer_loss:.4f}")

Epoch 1/14


Training Batches:   0%|          | 0/1501 [00:00<?, ?it/s]

RuntimeError: Sizes of tensors must match except in dimension 0. Expected size 712 but got size 706 for tensor number 1 in the list.

In [ ]:
test_df = pd.read_csv('/kaggle/input/ai-of-god-3/Public_data/test.csv')

In [ ]:
test_path = '/kaggle/input/ai-of-god-3/Public_data/test_images'

# Function to convert unique ID into correct image path
def construct_image_path(unique_id):
    parts = unique_id.split('_')
    page_number = parts[1]
    l_number = parts[3] 
    
    image_path = os.path.join(test_path, f'Page_{page_number}', f'L_{l_number}.png')
    return image_path

test_df['image_path'] = test_df['unique Id'].apply(construct_image_path)

# Check the resulting dataframe
print(test_df[[ 'image_path']].head())

In [ ]:
predictions = []
model.eval()

with torch.no_grad():
    for _, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Inference"):
        image_path = row['image_path']
        
        # Preprocess the image using the defined function
        pixel_values = preprocess_image(image_path).to(device)  # Add batch dimension and move to device
        
        # Generate output
        outputs = model.generate(pixel_values)

        # Decode the predicted string
        predicted_string = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        # Append the results
        predictions.append({
            'unique Id': row['unique Id'],  # Ensure 'unique Id' is correctly named in the dataframe
            'prediction': predicted_string
        })

# Create DataFrame for predictions
predictions_df = pd.DataFrame(predictions)

In [ ]:
# predictions = []

# model.eval()

# with torch.no_grad():
#     for _, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Inference"):
#         image_path = row['image_path']
        
#         # Open and preprocess the image (convert to grayscale, normalize, etc.)
#         image = Image.open(image_path).convert("RGB")
        
#         # Apply the transformations (grayscale, normalize, and ToTensor)
#         transform = transforms.Compose([
#             transforms.Grayscale(num_output_channels=3),
#             transforms.ToTensor()
#         ])
        
#         image = transform(image)
        
#         # Process the image using the processor
#         pixel_values = processor(image, return_tensors="pt").pixel_values.to(device)
        
#         # Generate output
#         outputs = model.generate(pixel_values)

#         # Decode the predicted string
#         predicted_string = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
#         # Append the results
#         predictions.append({
#             'unique Id': row['unique Id'],  # Ensure 'unique Id' is correctly named in the dataframe
#             'prediction': predicted_string
#         })

# # Create DataFrame for predictions
# predictions_df = pd.DataFrame(predictions)

In [ ]:
predictions_df.to_csv('submission.csv', index=False)

In [ ]:
predictions_df.head()